In [1]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from utils import exportsDataPath, workingDataPath, writeHypnoDfParquet

import pandas as pd
import datetime
import pytz
import json
print(pd.__version__)
utc = pytz.utc
AZtz = pytz.timezone("US/Arizona")

deviceName = 'fitbit'


exportPath = exportsDataPath + deviceName + \
    "/27-8-24/takeout-20240828T033834Z-001/Takeout/Fitbit/Global Export Data/"
files = [f for f in os.listdir(exportPath) if f.split("-")[0] == "sleep"]

print(len(files))

1.1.2
53


In [2]:
def fitbitSleepStageToStandard(stageName):
    if stageName == "restless":
        return 1
    if stageName == "asleep":
        return 2
    if stageName == "awake":
        return 0
    if stageName == "wake":
        return 0
    if stageName == "light":
        return 1
    if stageName == "deep":
        return 2
    if stageName == "rem":
        return 3
    print(stageName)
    return -1


In [3]:
data = []

for sf in files:
  with open(exportPath + sf) as f:
    jsonData = json.load(f)
  
  for sleepSession in jsonData:
        #print(sleepSession['levels']['data'])
        #print(sleepSession)
        
        for sleepStage in sleepSession['levels']['data']:
            row = []
            stageTime = AZtz.localize(pd.to_datetime(sleepStage['dateTime'], format='%Y-%m-%dT%H:%M:%S.%f'))
            row.append(stageTime)
            row.append(stageTime + datetime.timedelta(seconds=int(sleepStage['seconds'])))
            row.append(fitbitSleepStageToStandard(sleepStage['level']))
            data.append(row)
  
print(len(data))

unknown
45712


In [4]:
data = sorted(data, key=lambda x:x[0])
data[:2]

[[Timestamp('2020-05-13 13:43:00-0700', tz='US/Arizona'),
  Timestamp('2020-05-13 13:45:00-0700', tz='US/Arizona'),
  1],
 [Timestamp('2020-05-13 13:45:00-0700', tz='US/Arizona'),
  Timestamp('2020-05-13 14:41:00-0700', tz='US/Arizona'),
  2]]

In [5]:
# add in the no data segments
filledRecords = []
# for every row except the last
for ri in range(len(data)-1):
    filledRecords.append(data[ri])
    # if there is a gap between this rows end date and the next rows start date
    if data[ri][1] < data[ri+1][0]:
        filledRecords.append([
            data[ri][1],
            data[ri+1][0],
            -1
        ])
filledRecords.append(data[-1])
print(len(filledRecords))

47895


In [6]:
set([type(x[1]) for x in filledRecords])

{pandas._libs.tslibs.timestamps.Timestamp}

In [7]:
ColumnNames = ["startDate", "endDate", "value"]


fitBitHypnoDf = pd.DataFrame(columns=ColumnNames, data=filledRecords)

fitBitHypnoDf = fitBitHypnoDf.set_index("startDate")
fitBitHypnoDf

,endDate,value
startDate,,
2020-05-13 13:43:00-07:00,2020-05-13 13:45:00-07:00,1
2020-05-13 13:45:00-07:00,2020-05-13 14:41:00-07:00,2
2020-05-13 14:41:00-07:00,2020-05-13 14:42:00-07:00,1
2020-05-13 14:42:00-07:00,2020-05-13 14:43:00-07:00,2
2020-05-13 14:43:00-07:00,2020-05-14 01:21:00-07:00,-1
...,...,...
2024-08-27 12:20:30-07:00,2024-08-27 12:28:30-07:00,2
2024-08-27 12:28:30-07:00,2024-08-27 12:40:30-07:00,1
2024-08-27 12:40:30-07:00,2024-08-27 12:46:30-07:00,2


In [8]:
fitBitHypnoDf.dtypes

endDate    datetime64[ns, US/Arizona]
value                           int64
dtype: object

In [9]:
writeHypnoDfParquet(deviceName, fitBitHypnoDf)